# Image Classification using ResNet50

First, let's install and import the necessary libraries.

In [1]:
!pip install torch torchvision pillow requests

In [2]:
import torch
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO

Load and preprocess the image

In [3]:
# Define image transformations
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load image
image = Image.open('microscope_cat.jpg')
input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0)

Load the pre-trained ResNet50 model and run inference

In [4]:
# Load model and weights
weights = models.ResNet50_Weights.IMAGENET1K_V1
model = models.resnet50(weights=weights)
model.eval()

# Run inference
with torch.no_grad():
    output = model(input_batch)

# Get class names
categories = weights.meta['categories']

# Get top 5 predictions
_, indices = torch.sort(output[0], descending=True)
probabilities = torch.nn.functional.softmax(output[0], dim=0)

# Save results
with open('classification_results.txt', 'w') as f:
    f.write('Top 5 predictions:\n')
    for idx in indices[:5]:
        f.write(f'{categories[idx]}: {probabilities[idx].item():.3f}\n')